In [ ]:
!pip3 install geopandas

In [ ]:
!pip3 install uszipcode

In [6]:
import geopandas
from uszipcode import SearchEngine, SimpleZipcode, Zipcode
search = SearchEngine()

ModuleNotFoundError: No module named 'geopandas'

In [4]:
import json

from collections import defaultdict
import os


In [ ]:
myshpfile = geopandas.read_file('parking_facilities/parking_facilities.shp')

In [ ]:
myshpfile.shape

In [ ]:
obj = json.loads(myshpfile.geometry.to_json())

In [ ]:
len(obj['features'])

In [ ]:
#for points
out = defaultdict(list)
leftovers = []
for feature in obj['features']:
    x= feature['geometry']['coordinates'][0][0]
    path = dict(lat=x[1], lng=x[0])
    try:
        result = search.by_coordinates(path['lat'], path['lng'], radius=3)
    except:
        print(path)
    if result:
        if path not in out[result[0].zipcode]:
            out[result[0].zipcode].append(path)
    else:
        leftovers.append(path)
print(len(leftovers))

In [ ]:
with open('curb_ramps.json', 'wt') as f:
    f.write(json.dumps(out))

In [ ]:
#for lines
out = defaultdict(list)
leftovers = []
for feature in obj['features']:
    path = [dict(lat=x[1], lng=x[0]) for x in feature['geometry']['coordinates']]
    result = search.by_coordinates(path[0]['lat'], path[0]['lng'], radius=3)
    if result:
        out[result[0].zipcode].append(path)
    else:
        leftovers.append(path)
print(len(leftovers))

In [ ]:
#for polygons
out = defaultdict(list)
leftovers = []
for feature in obj['features']:
    path = []
    for i,f in enumerate(feature['geometry']['coordinates']):
        segment = []
        for x in f:
            if isinstance(x[0],list):
                print('hey')
                subsegment = []
                for i in x:
                    subsegment.append(dict(lat=i[1],lng=i[0]))
                segment.append(subsegment)
            else:
                segment.append(dict(lat=x[1],lng=x[0]))
        path.append(segment)
        try:
            result = search.by_coordinates(segment[0]['lat'], segment[0]['lng'], radius=3)
        except:
            result = search.by_coordinates(segment[0][0]['lat'], segment[0][0]['lng'], radius=3)
    if result:
        out[result[0].zipcode].append(path)
    else:
        leftovers.append(path)
print(len(leftovers))

In [ ]:
with open('facilities.json', 'wt') as f:
    f.write(json.dumps(out))

In [3]:
import pandas as pd

In [24]:
def get_zip_objects(df, zipc):
    print(df.shape)
    print(pd.unique(df.zipcode))
    print(zipc)
    slc = df.loc[df.zipcode == zipc,]
    print(slc.shape)
    classes = []
    for i in range(6):
        gen = slc.loc[slc['class']==i,].groupby(['new_lat', 'new_long']).apply(len).index.tolist()
        classes.append([(lat,lng) for lat,lng in gen])
    return classes

objects = pd.read_csv('/opt/w210/W210Capstone-AccessibleParkingMap/ui_v2/middleware/initial_20200630.csv').drop_duplicates()

In [ ]:
get_zip_objects(objects, 80264)

In [5]:
path='W210Capstone-AccessibleParkingMap/ui_v2/middleware/google_thresh/'

In [6]:
out = pd.DataFrame()
for f in os.listdir(path):
    out = pd.concat([out, pd.read_csv(path+f)])

In [21]:
out.columns


Index(['imgname', 'lat', 'long', 'angle', 'class', 'confidence', 'xcenter',
       'ycenter', 'width', 'hight', 'detect_loc', 'new_lat', 'new_long',
       'coord', 'zipcode'],
      dtype='object')

In [28]:
get_zip_objects(out, 80205)

(32218, 15)
[80262 80266 80110 80264 80271 80113 80202 80216 80203 80214 80002 80210
 80204 80238 80239 80205 80211 80207 80212 80206 80223 80237 80236 80222
 80220 80221 80209 80231 80219 80218 80224 80230 80226 80232 80227 80243
 80246 80290]
80205
(920, 15)


[[(39.74350666, -104.959048),
  (39.74379373, -104.9569028),
  (39.74379824, -104.9571474),
  (39.74382005, -104.9582261),
  (39.74382672, -104.9569895),
  (39.74385569, -104.95837079999998),
  (39.74390744, -104.9585099),
  (39.74399811, -104.9589361),
  (39.74412385, -104.9589457),
  (39.7442987, -104.95911509999998),
  (39.74449181, -104.9592655),
  (39.74450152, -104.9569267),
  (39.74475714, -104.9597652),
  (39.7448094, -104.9597061),
  (39.74621604, -104.9765374),
  (39.74628167, -104.978724),
  (39.74692039000001, -104.9775431),
  (39.74699652, -104.9708729),
  (39.7470583, -104.9684317),
  (39.747060600000005, -104.9787212),
  (39.747296780000006, -104.9722622),
  (39.7473218, -104.9517211),
  (39.74738614, -104.9609995),
  (39.74744097, -104.9746362),
  (39.74744776, -104.9684301),
  (39.74745688, -104.9747412),
  (39.74750533, -104.978721),
  (39.74750606, -104.9515607),
  (39.74764066, -104.9783773),
  (39.74764465, -104.9781708),
  (39.7476756, -104.9609969),
  (39.7477005

In [9]:
import math

In [26]:
def squared_linear_distance(lat, long, center): 
    return (lat - center['lat'])**2 + (long - center['lng'])**2

In [16]:
center={"lat": 39.7353602, "lng": -104.986306}

In [33]:

out['dist'] = squared_linear_distance(out['new_lat'], out['new_long'], center)

In [39]:
out.sort_values(by='dist', ascending=True)

,imgname,lat,long,angle,class,confidence,xcenter,ycenter,width,hight,detect_loc,new_lat,new_long,coord,zipcode,dist
751,39.73526955--104.98642109999999-0.jpg,39.735270,-104.986421,0,0,0.520975,0.920313,0.435156,0.125000,0.439063,R,39.735270,-104.986321,39.73526955-104.98642109999999,80203,8.445433e-09
743,39.73527316--104.98617639999999-0.jpg,39.735273,-104.986176,0,0,0.903947,0.189844,0.445312,0.114063,0.431250,L,39.735273,-104.986276,39.73527316-104.98617639999999,80203,8.452122e-09
750,39.73526955--104.98642109999999-0.jpg,39.735270,-104.986421,0,0,0.666504,0.515625,0.589063,0.028125,0.228125,C,39.735270,-104.986421,39.73526955-104.98642109999999,80203,2.146543e-08
475,39.73539801--104.98612340000001-270.jpg,39.735398,-104.986123,270,0,0.722438,0.620313,0.567969,0.028125,0.139062,C,39.735398,-104.986123,39.73539801-104.98612340000001,80203,3.477236e-08
474,39.73539801--104.98612340000001-270.jpg,39.735398,-104.986123,270,0,0.935240,0.126563,0.478125,0.128125,0.368750,L,39.735298,-104.986123,39.73539801-104.98612340000001,80203,3.721036e-08
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
42,39.62517993--104.9024929-90.jpg,39.625180,-104.902493,90,0,0.546608,0.878125,0.331250,0.171875,0.646875,R,39.625080,-104.902493,39.62517993-104.9024929,80237,1.918637e-02
62,39.62454631--104.90212290000001-90.jpg,39.624546,-104.902123,90,0,0.612006,0.953125,0.368750,0.093750,0.606250,R,39.624446,-104.902123,39.62454631-104.90212290000001,80237,1.938869e-02
38,39.62433467--104.90200039999999-90.jpg,39.624335,-104.902000,90,0,0.577073,0.533594,0.347656,0.085938,0.695312,C,39.624335,-104.902000,39.62433467-104.90200039999999,80237,1.943410e-02
20,39.62433467--104.90200039999999-270.jpg,39.624335,-104.902000,270,0,0.653001,0.257812,0.493750,0.046875,0.262500,L,39.624235,-104.902000,39.62433467-104.90200039999999,80237,1.945632e-02


In [44]:
PATH = 'W210Capstone-AccessibleParkingMap/ui_v2/middleware/'

In [45]:
with open(PATH+'curb_ramps.json', 'rt') as f:
    ramps = json.loads(f.read())

with open(PATH+'facilities.json', 'rt') as f:
    facilities = json.loads(f.read())

In [ ]:
out=out[['zipcode', 'new_lat', 'new_long', 'class']]

In [52]:
ramps_transform = []
for k,v in ramps.items():
    for r in v:
        rec = {'zipcode':k, 'new_lat':r['lat'], 'new_long':r['lng'], 'class':'ramp'}
        ramps_transform.append(rec)


In [54]:
pd.DataFrame(ramps_transform).shape

(59870, 4)

In [55]:
ramps_transform

[{'zipcode': '80230',
  'new_lat': 39.73863741679844,
  'new_long': -104.8869272588849,
  'class': 'ramp'},
 {'zipcode': '80230',
  'new_lat': 39.73864002580797,
  'new_long': -104.88475398198409,
  'class': 'ramp'},
 {'zipcode': '80230',
  'new_lat': 39.7387682572741,
  'new_long': -104.88591873021846,
  'class': 'ramp'},
 {'zipcode': '80230',
  'new_lat': 39.73876464126096,
  'new_long': -104.88575226061329,
  'class': 'ramp'},
 {'zipcode': '80230',
  'new_lat': 39.73864033480908,
  'new_long': -104.8857547696224,
  'class': 'ramp'},
 {'zipcode': '80230',
  'new_lat': 39.737330175046054,
  'new_long': -104.88637554187915,
  'class': 'ramp'},
 {'zipcode': '80230',
  'new_lat': 39.737328556040154,
  'new_long': -104.88647393223687,
  'class': 'ramp'},
 {'zipcode': '80230',
  'new_lat': 39.737221823652135,
  'new_long': -104.88648847828972,
  'class': 'ramp'},
 {'zipcode': '80230',
  'new_lat': 39.737221880652385,
  'new_long': -104.88637626288181,
  'class': 'ramp'},
 {'zipcode': '8023

In [57]:
facilities['80264']

[[[{'lat': 39.735360298000046, 'lng': -104.98630657599995},
   {'lat': 39.73537329100009, 'lng': -104.98723985999993},
   {'lat': 39.73590173900004, 'lng': -104.98723599699991},
   {'lat': 39.73619687000007, 'lng': -104.98723917299992},
   {'lat': 39.736303973000076, 'lng': -104.9871690519999},
   {'lat': 39.73630459100008, 'lng': -104.98628810399993},
   {'lat': 39.736336134000055, 'lng': -104.98628164999991},
   {'lat': 39.73634019100007, 'lng': -104.98618916999993},
   {'lat': 39.735959663000074, 'lng': -104.98619729099994},
   {'lat': 39.73598673500004, 'lng': -104.98626731899992},
   {'lat': 39.73599319400006, 'lng': -104.98672507599991},
   {'lat': 39.73562190800004, 'lng': -104.98672779299994},
   {'lat': 39.73562439400007, 'lng': -104.98627530699991},
   {'lat': 39.735360298000046, 'lng': -104.98630657599995}]],
 [[{'lat': 39.73631564600004, 'lng': -104.98634390099994},
   {'lat': 39.736317952000036, 'lng': -104.9868724669999},
   {'lat': 39.736383484000044, 'lng': -104.9868719

In [70]:
fac_transform = []
for k,v in facilities.items():
    for r in v:
        try:
            rec = {'zipcode':k, 'new_lat':r[0][0][0]['lat'], 'new_long':r[0][0][0]['lng'], 'class':7, 'path':r}
        except:
            rec = {'zipcode':k, 'new_lat':r[0][0]['lat'], 'new_long':r[0][0]['lng'], 'class':7, 'path':r}
            
        
        fac_transform.append(rec)

In [71]:
fac_transform[0]

{'zipcode': '80264',
 'new_lat': 39.735360298000046,
 'new_long': -104.98630657599995,
 'class': 7,
 'path': [[{'lat': 39.735360298000046, 'lng': -104.98630657599995},
   {'lat': 39.73537329100009, 'lng': -104.98723985999993},
   {'lat': 39.73590173900004, 'lng': -104.98723599699991},
   {'lat': 39.73619687000007, 'lng': -104.98723917299992},
   {'lat': 39.736303973000076, 'lng': -104.9871690519999},
   {'lat': 39.73630459100008, 'lng': -104.98628810399993},
   {'lat': 39.736336134000055, 'lng': -104.98628164999991},
   {'lat': 39.73634019100007, 'lng': -104.98618916999993},
   {'lat': 39.735959663000074, 'lng': -104.98619729099994},
   {'lat': 39.73598673500004, 'lng': -104.98626731899992},
   {'lat': 39.73599319400006, 'lng': -104.98672507599991},
   {'lat': 39.73562190800004, 'lng': -104.98672779299994},
   {'lat': 39.73562439400007, 'lng': -104.98627530699991},
   {'lat': 39.735360298000046, 'lng': -104.98630657599995}]]}

In [72]:
#TUESDAY AM!!! DO FOR m_streets and sidewalks, then push??

In [73]:
with open(PATH+'sidewalks.json', 'rt') as f:
    sidewalks = json.loads(f.read())

with open(PATH+'moratorium_streets.json', 'rt') as f:
    m_streets = json.loads(f.read())

In [78]:
path_transform = []
for k,v in sidewalks.items():
    for r in v:
        rec = {'zipcode':k, 
               'start_lat':r[0]['lat'], 
               'start_long':r[0]['lng'], 
               'end_lat': r[-1]['lat'],
               'end_long': r[-1]['lng'],
               'class':'sw', 
               'path':r}
        path_transform.append(rec)
for k,v in m_streets.items():
    for r in v:
        rec = {'zipcode':k, 
               'start_lat':r[0]['lat'], 
               'start_long':r[0]['lng'], 
               'end_lat': r[-1]['lat'],
               'end_long': r[-1]['lng'],
               'class':'mst', 
               'path':r}
        path_transform.append(rec)
paths = pd.DataFrame(path_transform)